In [1]:
import concurrent.futures
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import time
from image_gen import pick_random_image_path, generate_white_noise_image

In [2]:
# load a pre-trained ResNet model
model = models.resnet18(pretrained=True)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

C:\Users\muSyR\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\muSyR\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
high_res_image_path = pick_random_image_path("high-res-images")
print(f"Choosing high res image {high_res_image_path[-5]}.")

low_res_image_path = pick_random_image_path("low-res-images")
print(f"Choosing low res image {low_res_image_path[-5]}.")

white_noise_image_path = generate_white_noise_image()
if white_noise_image_path:
    print("Successfully generated white noise image.")

Choosing high res image 3.
Choosing low res image 2.
Successfully generated white noise image.


In [4]:
image_paths = [high_res_image_path, low_res_image_path, white_noise_image_path]

In [5]:
def classify_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
    return output.argmax().item()

In [6]:
# Function to process images using ThreadPoolExecutor
def process_with_threadpool():
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(classify_image, image_paths))
    end_time = time.time()
    total_time = end_time - start_time
    print(f"ThreadPoolExecutor results: {results}")
    # print(f"ThreadPoolExecutor total time: {total_time:.4f} seconds")
    return total_time

# Function to process images using ProcessPoolExecutor
def process_with_processpool():
    start_time = time.time()
    try: 
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = list(executor.map(classify_image, image_paths))
    except Exception as e:
        print(f"Error with Process Pool Execution: {e}")
        results = False
    end_time = time.time()
    total_time = end_time - start_time
    print(f"ProcessPoolExecutor results: {results}")
    # print(f"ProcessPoolExecutor total time: {total_time:.4f} seconds")
    return total_time

In [7]:
threadpool_time = process_with_threadpool()
processpool_time = process_with_processpool()

print(f"\nOverall time comparison:")
print(f"ThreadPoolExecutor Time: {threadpool_time:.4f} seconds")
print(f"ProcessPoolExecutor Time: {processpool_time:.4f} seconds")

ThreadPoolExecutor results: [669, 646, 418]
Error with Process Pool Execution: A process in the process pool was terminated abruptly while the future was running or pending.
ProcessPoolExecutor results: False

Overall time comparison:
ThreadPoolExecutor Time: 0.3089 seconds
ProcessPoolExecutor Time: 0.2473 seconds
